In [1]:
import pandas as pd
import uuid

In [2]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [4]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')
df_variables = df_variables.dropna(subset=['VAR-ID'])

In [5]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [6]:
#Poblar dimensión de etapas
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [7]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

In [8]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

In [9]:
#Poblar dimension variable

ids_vars = [str(uuid.uuid4()) for _ in range(len(df_variables))]

for index,row in df_variables.iterrows():
    if (index<634):
        variable.loc[index]=[
            ids_vars[index],
            row['NOMBRE EN LA BdeD'],
            row['VAR-ID'],
            row['UNITS'],
            False, #TODO: Edad coregida:De donde se saca esto?
            False, #TODO: Derivada: De donde se saca esto?
            False, #TODO: Impacto: ¿?
            row['VAR-TYPE'],
            False, #Tipo Variable
            False, #Nivel medicion
            False, #Longitudinal
            row['VAR-SHORT DESCRIPTION'],
            row['VAR-LONG DESCRIPTION']
        ]

In [10]:
#Crear diccionario de valor categorias
cols = [
    "VAL1", "SHORT-NAME1", "VAL2", "SHORT-NAME2",
    "VAL3", "SHORT-NAME3", "VAL4", "SHORT-NAME4", "VAL5", "SHORT-NAME5",
    "VAL6", "SHORT-NAME6", "VAL7", "SHORT-NAME7", "VAL8", "SHORT-NAME8",
    "VAL9", "SHORT-NAME9", "VAL10", "SHORT-NAME10", "VAL11", "SHORT-NAME11"
]

df_categorias=df_variables[cols]

unique_dicts = set()

def add_unique_dict(d):
    frozen = frozenset(d.items())
    if frozen not in unique_dicts:
        unique_dicts.add(frozen)

for index, row in df_categorias.iterrows():
    for i, col in  enumerate(df_categorias.columns):
        
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
            dicc={
                'value':int(row[col]),
                'description':str(row[siguiente_col]).strip()
            }
            add_unique_dict(dicc)


In [11]:
valorCategoria=pd.DataFrame(columns=['id','codificacion','descripcion'])
for d in unique_dicts:
    data = dict(d) 
    nueva_fila = {
        'id': str(uuid.uuid4()),               
        'codificacion': data['value'],         
        'descripcion': data['description']     
    }
    valorCategoria.loc[len(valorCategoria)] = nueva_fila

valorCategoria

,id,codificacion,descripcion
0,f35a487d-4124-4835-8c06-b617ba399f2b,6,Cambio de residencia
1,b4177984-28ed-408a-be47-ad659fee802c,2,SOLA CON APOYO
2,009ea0b7-667f-48f1-994d-4280ea23805b,7,Otros Diagnósticos
3,da9a711f-4c01-4c25-b5f8-9d4499530400,5,Cambio de departamento
4,75a7eb5f-79ce-43f4-b7b8-79c97bdafcc7,9,NO SE SABE
...,...,...,...
311,c066b1ad-f1a2-462f-bdf5-8e524121f299,2,de 1 a 2 SM percápita mensual
312,e46447b2-15b6-4cdd-bf6f-06fdbf5073cd,3,"SI, con CIRUGIA"
313,a4afebb7-70d2-40bd-ba69-5ec15d41e6d8,1,1er trimestre
314,f256933f-edf6-403b-a161-4536378de5c1,5,ROP grado 3+


In [12]:
#cargar archivo de variables longitudinales
file_dict=pd.ExcelFile('data/KMC-varLongitudinal.xlsx')
df_vLongitudinales=pd.read_excel(file_dict, sheet_name='VLs-Definition')

In [13]:
df_vLongitudinales.rename(columns={'Unnamed: 0': 'descripcion'}, inplace=True)

In [14]:
df_vLongitudinales

,descripcion,ERN,ENTRADA PosCang,40S,3M,6M,9M,12M
0,VL_Weight,ERN_Peso,V196A,V218,V261,V304,V347,V389
1,VL_Height,ERN_Talla,V196B,V219,V262,V305,V348,V390
2,VL_HeadCircunference,ERN_PC,V196C,V220,V263,V306,V349,V391
3,VL_WeightForHeight,pesotalla0,BMI1,BMI2,NaN,NaN,NaN,BMI12
4,VL_zscoreWeight,zscorepeso0,zscorepeso1,zscorepeso2,zscorepeso3,zscorepeso6,zscorepeso9,zscorepeso12
5,VL_zscoreHeight,zscoretalla0,zscoretalla1,zscoretalla2,zscoretalla3,zscoretalla6,zscoretalla9,zscoretalla12
6,VL_zscoreHeadCircunf,zscorePC0,zscorePC1,zscorePC2,zscorePC3,zscorePC6,zscorePC9,zscorePC12
7,VL_zscoreWeightForHeight,zscorepesotalla0,zscorepesotalla1,zscorepesotalla2,zscorepesotalla3,zscorepesotalla6,zscorepesotalla9,zscorepesotalla12
8,VL_Nutrition,-,alisalida,ali40,ali3m,ali6m,ali9m,ali12m
9,VL-SpeedZscoreWeight,-,-,-,velocidadzscore3m_40semOMS,velocidad6_3mesesOMS,velocidad9_6mesesOMS,velocidad12_9mesesOMS


In [15]:
#Creación bridge y grupo variable longitudial
bridgeVaribleLongitudinal= pd.DataFrame(columns=['idGrupoVariable','idVariableCompuesta','descripcion'])
grupoVariableLongitudinal = pd.DataFrame(columns=['idGrupo'])

In [16]:
#Poblar grupo y bridge variable longitudinal
ids_grupoVLs = [str(uuid.uuid4()) for _ in range(10)]

cont=0
for index, row in df_vLongitudinales.iterrows():
    for col in df_vLongitudinales.columns:
        if col!='descripcion':
            nombre_variable = row[col]  # Valor específico en esa celda
            var = variable[variable['nombreBD'] == nombre_variable]
            if not var.empty:
                id_variable = var.iloc[0]['id']
                bridgeVaribleLongitudinal.loc[len(bridgeVaribleLongitudinal)]=[
                ids_grupoVLs[cont],
                id_variable,
                str(row['descripcion'])+'-'+str(col)
            ]
            
    cont+=1

for i in range (10):
    grupoVariableLongitudinal.loc[len(grupoVariableLongitudinal)]=[
        ids_grupoVLs[i]
    ]

In [17]:
bridgeVaribleLongitudinal

,idGrupoVariable,idVariableCompuesta,descripcion
0,4d8d511c-08b7-4e49-8375-125a9a099bdf,9663f942-383f-44f3-8126-ddc7cfb51875,VL_Weight-ERN
1,4d8d511c-08b7-4e49-8375-125a9a099bdf,b1ff81fe-da79-4d72-9876-bb637266c6ad,VL_Weight-ENTRADA PosCang
2,4d8d511c-08b7-4e49-8375-125a9a099bdf,17183399-f9e5-45a4-863f-988e0283dca3,VL_Weight-40S
3,4d8d511c-08b7-4e49-8375-125a9a099bdf,80e73c79-c62b-41aa-ac0a-523622b5355e,VL_Weight-3M
4,4d8d511c-08b7-4e49-8375-125a9a099bdf,66d14034-8ae5-4d24-b260-efff2ed94502,VL_Weight-6M
5,4d8d511c-08b7-4e49-8375-125a9a099bdf,fe9cafcc-2eba-478e-a613-45499e497c55,VL_Weight-9M
6,4d8d511c-08b7-4e49-8375-125a9a099bdf,24058dc0-7dc5-4b6a-878f-abc4fafaea52,VL_Weight-12M
7,9d2ba905-4439-4315-bd1e-864002fcdaa2,ac97a985-1621-4ab5-a78c-2477d1aafcc0,VL_Height-ERN
8,9d2ba905-4439-4315-bd1e-864002fcdaa2,2d062591-7dd8-4691-beaf-f6c667346855,VL_Height-ENTRADA PosCang
9,9d2ba905-4439-4315-bd1e-864002fcdaa2,545cb382-bf9c-4332-aad4-b306273137ac,VL_Height-40S


In [18]:
#Creacion tabla de hechos variable sujeto 
hechoEvaluarVariableSujeto= pd.DataFrame(columns=['idVariable','idEtapa','idGrupoValorCategoria',
                                                  'idGrupoTemaInteres','idFechaModificacion','idFechaRegistro',
                                                  'idGrupoOperacion','idGrupoVariableLongitudinal'])

In [19]:
#Retorna la lista de ids de categorias relacionadas a  una fila (variable)
def definirGrupoIdValorCategoria(row):
    row=row[cols] #solo se tiene en cuenta las columnas de la codificación de categoria
    id=[]
    for i, col in  enumerate(df_categorias.columns):           
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
                val=valorCategoria[
                    (valorCategoria['codificacion']==row[col]) &
                    (valorCategoria['descripcion']==row[siguiente_col])
                ]
                if not val.empty:
                    id.append(str(val['id'].iloc[0]))
                
    return id

In [20]:
#Creacion bridge tema de interes
bridgeTemaInteres= pd.DataFrame(columns=['idGrupoTemaInteres','idTemaInteres'])

In [21]:
#Poblar tabla de hechos 
bridgeCategoria=pd.DataFrame(columns=['idGrupoValorCategoria','idValorCategoria'])

ids_grupoCats = [str(uuid.uuid4()) for _ in range(len(variable))]
ids_grupoTemas= [str(uuid.uuid4()) for _ in range(len(variable))]

for index,row in df_variables.iterrows():
    #id variable
    row_var = variable[
        variable['nombreAnalisis']==
        str(row['VAR-ID'])
    ]  
    var= str(row_var['id'].iloc[0])
    
    #id etapa
    row_etapa=etapa[etapa['numEtapa']==(row['#ETAPA'])]
    id_etapa=str(row_etapa['id'].iloc[0])

    #id categoria y grupo
    grupo_categoria=ids_grupoCats[index]
    ids_valores_cats=definirGrupoIdValorCategoria(row)
    
    #Poblar dimensión bridge categoria
    for i in ids_valores_cats:
        bridgeCategoria.loc[len(bridgeCategoria)]=[
            grupo_categoria,
            i
        ]

    #id tema interes
    row_tema1=temaInteres[temaInteres['nombre']==row['id-TEMA DE INTERES1']]
    row_tema2=temaInteres[temaInteres['nombre']==row['ID-TEMA DE INTERES2']]
    grupoTemaInteres=ids_grupoTemas[index]

    if not row_tema1.empty:
        bridgeTemaInteres.loc[len(bridgeTemaInteres)]=[
            grupoTemaInteres,
            str(row_tema1['id'].iloc[0])
        ]

    if not row_tema2.empty:
        bridgeTemaInteres.loc[len(bridgeTemaInteres)]=[
            grupoTemaInteres,
            str(row_tema2['id'].iloc[0])
        ]
  
    
    #id fecha modificación y registro
    fecha='1,1,2025'

    #grupo variable longitudinal
    row_vl=bridgeVaribleLongitudinal[bridgeVaribleLongitudinal['idVariableCompuesta']==var]
    if not row_vl.empty:
        id_longitudinal= row_vl.iloc[0]['idGrupoVariable']
    else:
        id_longitudinal= None

    id_operacion=None 
    
    hechoEvaluarVariableSujeto.loc[index]=[
        var,
        id_etapa,
        grupo_categoria,
        grupoTemaInteres,
        fecha,
        fecha,
        id_operacion,
        id_longitudinal
    ]

hechoEvaluarVariableSujeto


,idVariable,idEtapa,idGrupoValorCategoria,idGrupoTemaInteres,idFechaModificacion,idFechaRegistro,idGrupoOperacion,idGrupoVariableLongitudinal
0,d326ca08-54b0-4a97-bc3b-918826cab14a,e2537e47-87fe-42ad-a52c-040de4971d69,fbf3b626-de50-4c79-8b96-d6de92cb6acd,9ab9b563-494b-4287-abfb-8a2ad84634fd,"1,1,2025","1,1,2025",None,None
1,050649ad-d2a5-4210-bfaf-8454a65f7597,e2537e47-87fe-42ad-a52c-040de4971d69,980d768b-e2ca-4ff4-9886-7432e0be35d1,ee115ded-81d2-4469-8e83-7aef080be611,"1,1,2025","1,1,2025",None,None
2,f162a577-b50f-4cb6-87fd-ad9c0c2c5899,e2537e47-87fe-42ad-a52c-040de4971d69,4e3c266b-b254-4fbe-a8a3-a81b82072ee9,7d1d71c9-48fd-4477-8151-e39377523dcf,"1,1,2025","1,1,2025",None,None
3,533b7d15-d230-47ac-90ce-155c7fbbff38,e2537e47-87fe-42ad-a52c-040de4971d69,2607b973-eb20-4ecd-a0fd-1e95066e6418,9ac7593e-c764-408d-b9f7-1fe7cba53727,"1,1,2025","1,1,2025",None,None
4,5827168e-1746-4f30-9d31-fafab1358514,e2537e47-87fe-42ad-a52c-040de4971d69,4e9ad506-95ec-478c-b1cf-1f7df2068226,c972250c-6baf-4946-9580-8c44d8cfd018,"1,1,2025","1,1,2025",None,None
...,...,...,...,...,...,...,...,...
629,8675cfde-3c8a-433c-9b4d-9aabc3515983,d1fe6fdd-f0e9-4acc-a137-6f8a85b80283,6f897fe1-0de4-432e-9626-48e03764ac51,e622aa06-a8ae-465c-94e4-7b58da24b2f7,"1,1,2025","1,1,2025",None,None
630,8424b245-fead-407a-8107-3813fe09602d,d1fe6fdd-f0e9-4acc-a137-6f8a85b80283,0633bd1e-b22e-4413-8bc3-2b558a6a80a5,5c2e4ae0-8979-4f57-b243-e380db1817d4,"1,1,2025","1,1,2025",None,None
631,4f165960-3332-4ec6-8424-7b0bc4593a54,d1fe6fdd-f0e9-4acc-a137-6f8a85b80283,e7042b74-cc6e-46e9-be51-7843a58b23e4,5b085a6e-8932-483b-92a1-2c09c8b555b3,"1,1,2025","1,1,2025",None,None
632,1aa7c545-332f-4f8f-9362-916f66867b7d,d1fe6fdd-f0e9-4acc-a137-6f8a85b80283,586a6016-c805-416a-a632-cf7a58f05f8e,61047ce3-ea32-4105-8d31-79374064091b,"1,1,2025","1,1,2025",None,None
